In [ ]:
import datetime
import pathlib
import warnings

import load_ribasim  # noqa: F401
import pandas as pd
import ribasim
import ribasim.nodes
from ribasim import Node
from ribasim.nodes import level_boundary, pump, tabulated_rating_curve
from shapely import Point

import peilbeheerst_model.ribasim_parametrization as ribasim_param
from peilbeheerst_model.add_storage_basins import AddStorageBasins
from peilbeheerst_model.controle_output import Control
from peilbeheerst_model.ribasim_feedback_processor import RibasimFeedbackProcessor

%reload_ext autoreload
%autoreload 2
warnings.filterwarnings("ignore")

## Define variables and model

#### Set Config

In [ ]:
waterschap = "SchielandendeKrimpenerwaard"
work_dir = pathlib.Path(f"../../../../../Ribasim_updated_models/{waterschap}/modellen/{waterschap}_parametrized")
ribasim_gpkg = work_dir.joinpath("database.gpkg")
path_ribasim_toml = work_dir.joinpath("ribasim.toml")
output_dir = work_dir.joinpath("results")

# Basin area percentage
regular_percentage = 10
boezem_percentage = 90
unknown_streefpeil = (
    0.00012345  # we need a streefpeil to create the profiles, Q(h)-relations, and af- and aanslag peil for pumps
)

# Forcing settings
start_time = "2024-01-01"
timestep_size = "d"
timesteps = 2
delta_crest_level = 0.1  # delta waterlevel of boezem compared to streefpeil till no water can flow through an outlet

default_level = 0.75  # default LevelBoundary level, similar to surrounding Maas

## Process the feedback form

In [ ]:
name = "Ron Bruijns (HKV)"
versie = "2024_12_1"

feedback_excel = pathlib.Path(f"../../../../../Ribasim_feedback/V1_formulieren/feedback_formulier_{waterschap}.xlsx")
feedback_excel_processed = (
    f"../../../../..//Ribasim_feedback/V1_formulieren_verwerkt/feedback_formulier_{waterschap}_JA_processed.xlsx"
)

ribasim_toml = f"../../../../../Ribasim_base_models/{waterschap}_boezemmodel_{versie}/ribasim.toml"
output_folder = work_dir  # f"../../../../../Ribasim_updated_models/{waterschap}"

processor = RibasimFeedbackProcessor(
    name, waterschap, versie, feedback_excel, ribasim_toml, output_folder, feedback_excel_processed, use_validation=True
)
processor.run()

#### Load model

In [ ]:
# Load Ribasim model
with warnings.catch_warnings():
    warnings.simplefilter(action="ignore", category=FutureWarning)
    ribasim_model = ribasim.Model(filepath=path_ribasim_toml)

In [ ]:
ribasim_model.pump.node.df.loc[ribasim_model.pump.node.df.index == 659]

In [ ]:
ribasim_model.tabulated_rating_curve.node.df.loc[ribasim_model.tabulated_rating_curve.node.df.index == 659]

# Parameterization

## Nodes

### Basin (characteristics)

In [ ]:
ribasim_param.validate_basin_area(ribasim_model)

In [ ]:
# remove the basins of above in the feedback form

## Model specific tweaks

In [ ]:
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1

In [ ]:
# change unknown streefpeilen to a default streefpeil
ribasim_model.basin.area.df.loc[
    ribasim_model.basin.area.df["meta_streefpeil"] == "Onbekend streefpeil", "meta_streefpeil"
] = str(unknown_streefpeil)
ribasim_model.basin.area.df.loc[ribasim_model.basin.area.df["meta_streefpeil"] == -9.999, "meta_streefpeil"] = str(
    unknown_streefpeil
)

In [ ]:
assert not pd.isnull(ribasim_model.basin.area.df.meta_streefpeil).any()

In [ ]:
# add the LevelBoundaries and Pumps
# 1
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(89792, 437301)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(89992, 437341)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[116], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 2
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(92237, 435452)), [level_boundary.Static(level=[default_level])]
)
tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(
    Node(new_node_id + 1, Point(92144, 435533)),
    [tabulated_rating_curve.Static(level=[0.0, 0.1234], flow_rate=[0.0, 0.1234])],
)
ribasim_model.edge.add(level_boundary_node, tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, ribasim_model.basin[110])

# 3
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(95111, 436428)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(95431, 436488)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[157], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 4
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(95111, 436418)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(95736, 436211)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[143], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 5 not sure whether this one is correct. FF was not clear
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(94966, 437011)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(94966, 437011)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[25], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 6
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(92476, 435889)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(92334, 436428)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[142], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 7
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(104547, 443432)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(104485, 443506)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[19], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 7
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(104555, 443434)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(104527, 443455)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[2], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 8
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(102079, 438209)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(102230, 438082)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[29], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

# 9, added by RB as it is quiet clear a levelboundary is missing.
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(91451, 439245)), [level_boundary.Static(level=[default_level])]
)
tabulated_rating_curve_node = ribasim_model.tabulated_rating_curve.add(
    Node(new_node_id + 1, Point(91514, 439286)),
    [tabulated_rating_curve.Static(level=[0.0, 0.1234], flow_rate=[0.0, 0.1234])],
)
ribasim_model.edge.add(ribasim_model.basin[28], tabulated_rating_curve_node)
ribasim_model.edge.add(tabulated_rating_curve_node, level_boundary_node)

# 10. cooperation day 12 dec 2024: Erik suggested adding an additional pump which already exists, but this would prevent two entire basins interacting with eachother if only one basin would get a pump, while this pump discharges both basins
new_node_id = max(ribasim_model.edge.df.from_node_id.max(), ribasim_model.edge.df.to_node_id.max()) + 1
level_boundary_node = ribasim_model.level_boundary.add(
    Node(new_node_id, Point(89786, 437309)), [level_boundary.Static(level=[default_level])]
)

pump_node = ribasim_model.pump.add(Node(new_node_id + 1, Point(89985, 437345)), [pump.Static(flow_rate=[0.1])])

ribasim_model.edge.add(ribasim_model.basin[115], pump_node)
ribasim_model.edge.add(pump_node, level_boundary_node)

In [ ]:
ribasim_model.level_boundary.node.df.meta_node_id = ribasim_model.level_boundary.node.df.index
ribasim_model.tabulated_rating_curve.node.df.meta_node_id = ribasim_model.tabulated_rating_curve.node.df.index
ribasim_model.pump.node.df.meta_node_id = ribasim_model.pump.node.df.index

## Implement standard profile and a storage basin

In [ ]:
# Insert standard profiles to each basin. These are [depth_profiles] meter deep, defined from the streefpeil
ribasim_param.insert_standard_profile(
    ribasim_model,
    unknown_streefpeil=unknown_streefpeil,
    regular_percentage=regular_percentage,
    boezem_percentage=boezem_percentage,
    depth_profile=2,
)

In [ ]:
add_storage_basins = AddStorageBasins(
    ribasim_model=ribasim_model, exclude_hoofdwater=True, additional_basins_to_exclude=[]
)

add_storage_basins.create_bergende_basins()

### Basin (forcing)

In [ ]:
# Set static forcing
forcing_dict = {
    "precipitation": ribasim_param.convert_mm_day_to_m_sec(10 * 2),
    "potential_evaporation": ribasim_param.convert_mm_day_to_m_sec(0),
    "drainage": ribasim_param.convert_mm_day_to_m_sec(0),
    "infiltration": ribasim_param.convert_mm_day_to_m_sec(0),
    # 'urban_runoff':          ribasim_param.convert_mm_day_to_m_sec(0),
}

ribasim_param.set_static_forcing(timesteps, timestep_size, start_time, forcing_dict, ribasim_model)

### Pumps

In [ ]:
# Set pump capacity for each pump
ribasim_model.pump.static.df["flow_rate"] = 0.16667  # 10 kuub per minuut

### Convert all boundary nodes to LevelBoundaries

In [ ]:
ribasim_param.Terminals_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)  # clean
ribasim_param.FlowBoundaries_to_LevelBoundaries(ribasim_model=ribasim_model, default_level=default_level)

# test for better afwatering
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id == 728, "level"] = -1.3
ribasim_model.edge.df.loc[ribasim_model.edge.df.to_node_id == 728, "meta_categorie"] = "hoofdwater"
ribasim_model.edge.df.loc[ribasim_model.edge.df.to_node_id == 728, "meta_to_node_type"] = "LevelBoundary"

# test for better afwatering
ribasim_model.level_boundary.static.df.loc[ribasim_model.level_boundary.static.df.node_id == 638, "level"] = -0.1

### Add Outlet

In [ ]:
ribasim_param.add_outlets(ribasim_model, delta_crest_level=0.10)

## Add control, based on the meta_categorie

In [ ]:
ribasim_param.identify_node_meta_categorie(ribasim_model)

In [ ]:
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="outlet")
ribasim_param.find_upstream_downstream_target_levels(ribasim_model, node="pump")

In [ ]:
# ribasim_param.add_discrete_control(ribasim_model, waterschap, default_level)

In [ ]:
ribasim_param.determine_min_upstream_max_downstream_levels(ribasim_model, waterschap)

### Manning Resistance

In [ ]:
# there is a MR without geometry and without edges for some reason
ribasim_model.manning_resistance.node.df = ribasim_model.manning_resistance.node.df.dropna(subset="geometry")

In [ ]:
# lower the difference in waterlevel for each manning node
ribasim_model.manning_resistance.static.df.length = 10
ribasim_model.manning_resistance.static.df.manning_n = 0.01

## Last formating of the tables

In [ ]:
# only retain node_id's which are present in the .node table
ribasim_param.clean_tables(ribasim_model, waterschap)

# Set numerical settings

In [ ]:
# Write model output
ribasim_model.use_validation = True
ribasim_model.starttime = datetime.datetime(2024, 1, 1)
ribasim_model.endtime = datetime.datetime(2025, 1, 1)
ribasim_model.solver.saveat = 3600
ribasim_param.write_ribasim_model_Zdrive(ribasim_model, path_ribasim_toml)

## Run Model

In [ ]:
ribasim_param.tqdm_subprocess(["ribasim", path_ribasim_toml], print_other=False, suffix="init")

In [ ]:
controle_output = Control(work_dir=work_dir)
indicators = controle_output.run_all()

# Write model

In [ ]:
# control_dict = Control(work_dir = work_dir).run_all()
ribasim_param.write_ribasim_model_GoodCloud(
    ribasim_model=ribasim_model,
    path_ribasim_toml=path_ribasim_toml,
    waterschap=waterschap,
    modeltype="boezemmodel",
    include_results=True,
)